In [ ]:
from __future__ import unicode_literals, print_function, division
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from preprocessing import get_dataframes
train_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/simple_split/tasks_train_simple.txt"
test_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/simple_split/tasks_test_simple.txt"
train_df, test_df = get_dataframes(train_url, test_url)

train_df.head()

In [ ]:
from preprocessing import get_vocab

train_vocab_in = get_vocab(train_df['IN'], 'in')
train_vocab_out = get_vocab(train_df['OUT'], 'out')

print(train_vocab_in.word2index)
print(train_vocab_out.word2index)

def series2idx(row):
    x = train_vocab_in.col2idx(row, 'IN')
    y = train_vocab_out.col2idx(row, 'OUT')
    return x, y
    
train_df[['IN_idx','OUT_idx']] = train_df.apply(series2idx, axis=1, result_type='expand')
test_df[['IN_idx','OUT_idx']] = test_df.apply(series2idx, axis=1, result_type='expand')

train_df.head()

In [ ]:
from dataloading import Text_dataset
from torch.utils.data import DataLoader

# Create datasets
training_data = Text_dataset(train_df[['IN_idx', 'OUT_idx']], True, 100)
test_data = Text_dataset(test_df[['IN_idx', 'OUT_idx']], False)

# Define dataloaders
bs = 1 # Batch size
train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=False)

In [ ]:
import models
from training import train
max_len = train_df.OUT_idx.apply(len).max()

layers = 2
hidden_size = 200
dropout = 0

encoder = models.EncoderRNN('lstm', train_vocab_in.n_words, hidden_size, layers, dropout).to(device)
decoder = models.DecoderRNN('lstm', hidden_size, train_vocab_out.n_words, layers, device, max_len).to(device)

train(train_dataloader, encoder, decoder, device, save_name='ex1_1')

In [ ]:
layers = 2
hidden_size = 200
dropout = 0

encoder = models.EncoderRNN('lstm', train_vocab_in.n_words, hidden_size, layers, dropout).to(device)
decoder = models.DecoderRNN('lstm', hidden_size, train_vocab_out.n_words, layers, device, max_len).to(device)
encoder.load_state_dict(torch.load('models/encoder_ex1_1.pth'))
decoder.load_state_dict(torch.load('models/decoder_ex1_1.pth'))


In [ ]:
from training import evaluate

acc = evaluate(encoder, decoder, test_dataloader, device)

In [ ]:
acc